In [2]:
%reset -f
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from wavhandler import *
from utils import *
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import logging
logger = logging.getLogger()
logger.propagate = False
logger.setLevel(logging.ERROR)
np.random.seed(0)


df_mosquitos = pd.read_csv(TEMP_DATADIR+'mosquitos.csv', delimiter=';')

Using TensorFlow backend.


In [3]:
#bi_classes = ['LG_drosophila_10_09', 'LG_zapr_26_09']
#target_names = all_6
DATADIR = '/home/kalfasyan/data/insects/increasing dataset/'

# DATADIR = '/home/kalfasyan/data/insects/LG2/'
target_names = os.listdir(DATADIR)

X_names, y = get_data(filedir= DATADIR,
                      target_names=target_names, nr_signals=np.inf, only_names=True)

100%|██████████| 10/10 [00:00<00:00, 49.62it/s]


In [7]:
%%time
X = make_df_parallel(names=X_names, setting='psd_dB')
print(X.shape)

(129, 50000)
CPU times: user 7.34 s, sys: 369 ms, total: 7.7 s
Wall time: 14.3 s


In [8]:
df_concat = pd.DataFrame(X.T)
df_concat['label'] = y
print(df_concat.label.value_counts())
# df_concat = df_concat[df_concat.label.isin([target_names[0]])]
print(df_concat.label.value_counts())

X = df_concat.iloc[:,:-1].values
y = df_concat.label.values

9    5000
8    5000
7    5000
6    5000
5    5000
4    5000
3    5000
2    5000
1    5000
0    5000
Name: label, dtype: int64
9    5000
8    5000
7    5000
6    5000
5    5000
4    5000
3    5000
2    5000
1    5000
0    5000
Name: label, dtype: int64


In [9]:
X_std = StandardScaler(with_std=True).fit_transform(X);  # standardization of data

X, y = shuffle(X, y, random_state=3)
#del df

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2018)

In [10]:
x_train = np.expand_dims(x_train, axis=2)

In [11]:
# input_dim = x_train.shape[1]
# encoding_dim = 24  
# input_img = Input(shape=(input_dim,))
# encoded = Dense(64, activation='relu')(input_img)
# encoded = Dense(32, activation='relu')(encoded)
# # encoded = Dense(16, activation='relu')(encoded)
# # encoded = Dense(8, activation='relu')(encoded)
# encoded = Dense(encoding_dim, activation='relu')(encoded)
# # decoded = Dense(8, activation='relu')(encoded)
# # decoded = Dense(16, activation='relu')(decoded)
# decoded = Dense(32, activation='relu')(encoded)
# decoded = Dense(64, activation='relu')(decoded)
# decoded = Dense(input_dim, activation='softmax')(decoded)
# autoencoder = Model(input_img, decoded)
# autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
# print(autoencoder.summary())

In [12]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape
from keras.models import Model
from keras import backend as K

# ENCODER
input_sig = Input(batch_shape=(None,129,1))
x = Conv1D(64,3, activation='relu', padding='valid')(input_sig)
x1 = MaxPooling1D(2)(x)
x2 = Conv1D(32,3, activation='relu', padding='valid')(x1)
x3 = MaxPooling1D(2)(x2)
flat = Flatten()(x3)
encoded = Dense(32,activation = 'relu')(flat)
 
print("shape of encoded {}".format(K.int_shape(encoded)))
 
# DECODER 
x2_ = Conv1D(32, 3, activation='relu', padding='valid')(x3)
x1_ = UpSampling1D(2)(x2_)
x_ = Conv1D(64, 3, activation='relu', padding='valid')(x1_)
upsamp = UpSampling1D(2)(x_)
flat = Flatten()(upsamp)
decoded = Dense(129,activation = 'relu')(flat)
decoded = Reshape((129,1))(decoded)
 
print("shape of decoded {}".format(K.int_shape(decoded)))
 
autoencoder = Model(input_sig, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
shape of encoded (None, 32)
shape of decoded (None, 129, 1)


In [13]:
history = autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=32,
                shuffle=True,
                validation_split=0.2,
                verbose = 1)

Train on 32000 samples, validate on 8000 samples
Epoch 1/100
32000/32000 [==============================] - 21s 661us/step - loss: 10175.1542 - acc: 0.0000e+00 - val_loss: 10227.1903 - val_acc: 0.0000e+00
Epoch 2/100
32000/32000 [==============================] - 12s 364us/step - loss: 10174.6771 - acc: 0.0000e+00 - val_loss: 10227.1903 - val_acc: 0.0000e+00
Epoch 3/100
32000/32000 [==============================] - 11s 357us/step - loss: 10174.6771 - acc: 0.0000e+00 - val_loss: 10227.1903 - val_acc: 0.0000e+00
Epoch 4/100
32000/32000 [==============================] - 11s 356us/step - loss: 10174.6771 - acc: 0.0000e+00 - val_loss: 10227.1903 - val_acc: 0.0000e+00
Epoch 5/100
32000/32000 [==============================] - 11s 353us/step - loss: 10174.6771 - acc: 0.0000e+00 - val_loss: 10227.1903 - val_acc: 0.0000e+00
Epoch 6/100
 8320/32000 [======>.......................] - ETA: 7s - loss: 10156.6199 - acc: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
# use our encoded layer to encode the training input
encoder = Model(input_img, encoded)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[5]
decoder = Model(encoded_input, decoder_layer(encoded_input))
encoded_data = encoder.predict(x_test)
print(encoded_data.shape)

In [96]:
import matplotlib.pyplot as plt
sns.set()
plt.figure(figsize=(18,14))
sns.scatterplot(encoded_data[:,0], encoded_data[:,1], hue=y_test, style=y_test, alpha=0.8)

NameError: name 'encoded_data' is not defined

<Figure size 1296x1008 with 0 Axes>

In [ ]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.plot(x_test[i])#.reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.plot(decoded_imgs[i])#.reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# 0.0469 loss with bs=128 and encoding_dim=15 